In [8]:
from sympy.physics.paulialgebra import Pauli
import numpy as np
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import PauliSumOp, CircuitSampler, StateFn, PauliExpectation, PauliTrotterEvolution, Suzuki
from qiskit import Aer, QuantumCircuit
import time
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# 系统参数配置
n = 2  # 作业数
m_bits = 2  # 机器编码比特数
l = 2  # 时间变量比特数
num_qubits = n * m_bits + l

# 处理时间矩阵
processing_time = {
    0: {0: 1.5, 1: 2.0},
    1: {0: 1.0, 1: 3.0}
}

# 合法机器编码
valid_machines = {
    0: [0, 0],  # 机器0
    1: [1, 0]  # 机器1
}

# 非法机器编码
illegal_machines = [
    [0, 1]  # 非法机器01
]

# 量子比特数 = 作业分配(n*N) + C_max(l)
# num_qubits = n * N + l
from qiskit.quantum_info import SparsePauliOp
from collections import defaultdict


def build_hamiltonian():
    num_qubits = n * m_bits + l
    K1, P = 1000, 2000
    pauli_terms = defaultdict(float)

    # ----------------- 1. 非法分配惩罚项 -----------------
    for illegal in illegal_machines:
        for job in range(n):
            start = job * m_bits
            # 生成每个比特的约束项
            for bit in range(m_bits):
                q = start + bit
                target = illegal[bit]
                # 线性项: 0.5 - target
                pauli_str = ['I'] * num_qubits
                pauli_str[q] = 'Z'
                key = ''.join(pauli_str)
                pauli_terms[key] += K1 * (0.5 - target)
                # 二次项: 约束同一作业的不同比特
                for next_bit in range(bit + 1, m_bits):
                    q_next = start + next_bit
                    pauli_str_quad = ['I'] * num_qubits
                    pauli_str_quad[q] = 'Z'
                    pauli_str_quad[q_next] = 'Z'
                    key_quad = ''.join(pauli_str_quad)
                    pauli_terms[key_quad] += K1 * 0.25

    # ----------------- 2. 时间约束项 -----------------
    for mid, code in valid_machines.items():
        for job in range(n):
            start = job * m_bits
            p_ij = processing_time[job][mid]

            # 生成机器选择项
            machine_terms = defaultdict(float)
            for bit in range(m_bits):
                q = start + bit
                target = code[bit]
                # 线性项
                pauli_str = ['I'] * num_qubits
                pauli_str[q] = 'Z'
                key = ''.join(pauli_str)
                machine_terms[key] += (0.5 - target)
                # 二次项
                for next_bit in range(bit + 1, m_bits):
                    q_next = start + next_bit
                    pauli_str_quad = ['I'] * num_qubits
                    pauli_str_quad[q] = 'Z'
                    pauli_str_quad[q_next] = 'Z'
                    key_quad = ''.join(pauli_str_quad)
                    machine_terms[key_quad] += 0.25

            # 处理时间平方项
            for key, coeff in machine_terms.items():
                pauli_terms[key] += P * (p_ij ** 2) * coeff

            # 交叉项（与时间变量耦合）
            for h in range(l):
                z = n * m_bits + h
                coeff = -2 * P * p_ij * (2 ** h)
                for bit in range(m_bits):
                    q = start + bit
                    target = code[bit]
                    pauli_str = ['I'] * num_qubits
                    pauli_str[q] = 'Z'
                    pauli_str[z] = 'Z'
                    key = ''.join(pauli_str)
                    pauli_terms[key] += coeff * (0.5 - target)

    # ----------------- 3. C_max项 -----------------
    for h in range(l):
        z = n * m_bits + h
        pauli_str = ['I'] * num_qubits
        pauli_str[z] = 'Z'
        key = ''.join(pauli_str)
        pauli_terms[key] += 2 ** h + P * (2 ** (2 * h)) * n

    # ----------------- 转换为稀疏Pauli算符 -----------------
    # 合并同类项并过滤小系数
    filtered = {k: v for k, v in pauli_terms.items() if abs(v) > 1e-3}
    return SparsePauliOp.from_list([(k, v) for k, v in filtered.items()])


from qiskit.opflow import PauliTrotterEvolution, Suzuki


def _build_qaoa_circuit(self, layer, params):
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))

    # 分块参数
    chunk_size = 5  # 每块最多处理5个Pauli项

    for p in range(layer):
        gamma = params[2 * p]
        beta = params[2 * p + 1]

        # ========== 问题哈密顿量分块演化 ==========
        hamiltonian_list = self.hamiltonian.to_list()
        for i in range(0, len(hamiltonian_list), chunk_size):
            chunk = hamiltonian_list[i:i + chunk_size]
            chunk_op = SparsePauliOp.from_list(chunk) * gamma
            evolution = PauliTrotterEvolution(trotter_mode=Suzuki(order=1)).convert(chunk_op.exp_i())
            qc.append(evolution.to_circuit(), qc.qubits)

        # ========== 混合哈密顿量优化 ==========
        mixer_terms = [('X' + 'I' * (num_qubits - 1), 1)]  # 仅第一个量子比特作用X门
        mixer = SparsePauliOp.from_list(mixer_terms) * beta
        mixer_evolution = PauliTrotterEvolution(trotter_mode=Suzuki(order=1)).convert(mixer.exp_i())
        qc.append(mixer_evolution.to_circuit(), qc.qubits)

    qc.measure_all()
    return qc


# ================== QAOA优化器 ==================
class QAOAOptimizer:
    def __init__(self, hamiltonian, max_layers=3, trials=5):
        self.hamiltonian = hamiltonian
        self.max_layers = max_layers
        self.trials = trials
        self.optimal = {'params': [], 'energy': float('inf')}
        self.history = []

        # 初始化量子后端
        self.backend = Aer.get_backend('qasm_simulator')
        self.sampler = CircuitSampler(self.backend)

    def optimize(self):
        for layer in range(1, self.max_layers + 1):
            print(f"== 优化层数: {layer} ==")

            # 生成初始参数
            params_set = self._generate_initial_params(layer)

            # 遍历参数组合
            for params in params_set:
                energy = self._compute_energy(layer, params)

                # 更新最优结果
                if energy < self.optimal['energy']:
                    self.optimal.update({
                        'params': params,
                        'energy': energy
                    })
                    print(f" 发现更优解 | 能量: {energy:.4f} | 参数: {np.round(params, 3)}")

            # 早停检查
            if self._check_convergence():
                break

        return self.optimal

    def _generate_initial_params(self, layer):
        """生成初始参数，确保参数维度为2*layer"""
        params_set = []

        # 首层参数生成
        if layer == 1:
            return [np.random.uniform(0, 2 * np.pi, 2) for _ in range(self.trials)]

        # 后续层参数扩展
        for _ in range(self.trials):
            # 继承前一层参数并追加新参数
            base = list(self.optimal['params'])
            new_params = base + [np.random.uniform(0, 2 * np.pi) for _ in range(2)]
            params_set.append(new_params)

        return params_set

    def _compute_energy(self, layer, params):
        """计算给定参数的期望能量"""
        qc = self._build_qaoa_circuit(layer, params)
        expectation = StateFn(self.hamiltonian, is_measurement=True) @ StateFn(qc)
        measurable_exp = PauliExpectation().convert(expectation)
        return np.real(self.sampler.convert(measurable_exp).eval())

    def _build_qaoa_circuit(self, layer, params):
        """构建QAOA量子线路"""
        qc = QuantumCircuit(num_qubits)

        # 制备叠加态
        qc.h(range(num_qubits))

        # 应用各层操作
        for p in range(layer):
            gamma = params[2 * p]
            beta = params[2 * p + 1]

            # 问题哈密顿量演化
            problem_hamil = gamma * self.hamiltonian
            evolution = PauliTrotterEvolution(trotter_mode=Suzuki(order=1)).convert(problem_hamil.exp_i())
            qc.append(evolution.to_circuit(), qc.qubits)

            # 混合哈密顿量演化
            mixer = sum(PauliSumOp(Pauli(f"X{'I' * (num_qubits - 1)}")) for _ in range(num_qubits))
            mixer_hamil = beta * mixer
            mixer_evolution = PauliTrotterEvolution(trotter_mode=Suzuki(order=1)).convert(mixer_hamil.exp_i())
            qc.append(mixer_evolution.to_circuit(), qc.qubits)

        # 添加测量
        qc.measure_all()
        return qc

    def _check_convergence(self):
        """检查能量是否收敛"""
        if len(self.history) < 3:
            return False

        last_three = self.history[-3:]
        return abs(last_three[-1] - last_three[-2]) < 0.01 and \
            abs(last_three[-2] - last_three[-3]) < 0.01


# ================== 执行优化 ==================
if __name__ == "__main__":
    # 构建哈密顿量
    H = build_hamiltonian()
    # print(H)
    # 转换为泡利算符
    # pauli_terms = []
    # for i in range(num_qubits):
    #     for j in range(i, num_qubits):
    #         if H[i,j] != 0:
    #             pauli_str = ['I']*num_qubits
    #             pauli_str[i] = 'Z'
    #             if j != i: pauli_str[j] = 'Z'
    #             pauli_terms.append((''.join(pauli_str), H[i,j]))
    # hamiltonian = PauliSumOp.from_list(pauli_terms)
    # print(hamiltonian)
    # 执行QAOA优化
    optimizer = QAOAOptimizer(H, max_layers=3, trials=5)
    result = optimizer.optimize()

    print("\n==== 优化结果 ====")
    print(f"最优能量值: {result['energy']:.4f}")
    print(f"最优参数: {np.round(result['params'], 4)}")

== 优化层数: 1 ==


AttributeError: 'SparsePauliOp' object has no attribute 'exp_i'